<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Morpheme analyzer
*   [Reference](https://soohee410.github.io/compare_tagger)
*   [Install](https://sanghyu.tistory.com/170)

In [ ]:
# okt, komoran, kkma
# install konlpy (okt, komoran, kkma)
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

# mecab (take a long time)
# set env
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

# install konlpy (mecab)
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# Error Solution

1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings' [(JVM)](https://gyulogs.tistory.com/130)
2. NameError: name 'Tagger' is not defined [(Mecab)](https://sosomemo.tistory.com/31)
3. ParserError: Error tokenizing data. C error [(Pandas)](https://mskim8717.tistory.com/82)



In [ ]:
""" 
ERROR 1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings'
- Solution: /usr/local/lib/python3.7/dist-packages/konlpy/jvm.py, 67 line (convertStrings=True) comments processing and save jvm.py before import pakage
"""

""" 
ERROR 2. NameError: name 'Tagger' is not defined 
- Solution: Execute mecab.sh script (under code excute)
"""
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# Import
*   [KoNLPy Reperence](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)



In [ ]:
# import for MechineLearning
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# import Morpheme analyzer
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from konlpy.utils import pprint

# import etc
import time

%cd /content/drive/My Drive/DeepLearning/AI_KKH

# Funtion

In [ ]:
"""
Function about time comparison

def pos_times(taggers, tagger_name, texts) :
    time_list = []
    for tagger in taggers :
        print(type(tagger))
        
        t1 = time.time()
        for text in texts :
            pprint(tagger.pos(text))
        
        print("###################################")
        t2 = time.time()
        time_list.append(t2 - t1)

    plt.figure(figsize=(10,8))
    plt.bar(tagger_name, time_list, color=(0.4,0.7,0.5))
    plt.title('Learning Time with .pos', fontsize=17)
    plt.ylabel('total seconds')
"""
# Extract Data about kakao talk's content
def data_extraction(x_list, y_list) :
    for text in test.values :
        for t in text : 
            if "프로젝트 김대엽 클럽장" in t :
                ptext = t[(t.rfind(']') + 2) : len(t)]
                if ptext.find("http") == -1 and ptext != "" : y_list.append(ptext)
            elif "]" in t :
                ptext = t[(t.rfind(']') + 2) : len(t)]
                if ptext.find("http") == -1 and ptext != "" : x_list.append(ptext)

# Auto calibrate spacing function
def calibrate_spcaing(input_list) :
    for text in input_list :
        analyzedRes = mecab.pos(text)
        for mecabR in analyzedRes :
            if mecabR[1] == "MAG" :
                try :
                    startIdx = text.index(mecabR[0])
                    if len(mecabR[0]) == 1 :
                        if text[startIdx + 1] != " " and text[startIdx + 1].isalnum() : 
                            if text[startIdx + 2] != " " :
                                repairIdx = input_list.index(text)
                                input_list[repairIdx] = text[:startIdx] + text[startIdx] + " " + text[startIdx + 1 :]
                    else : 
                        if text[startIdx + len(mecabR[0])] != " " and text[startIdx + len(mecabR[0])].isalnum() : 
                            if text[startIdx + len(mecabR[0]) + 1] != " " :
                                repairIdx = input_list.index(text)
                                input_list[repairIdx] = text[:startIdx] + text[startIdx : startIdx + len(mecabR[0])] + " " + text[startIdx + len(mecabR[0]) :]
                except IndexError :
                    continue

#Main

In [ ]:
"""
ranking about time spent morpheme analyzing (The fastest is number one.)

1. Mecab
2. Komoran
3. Okt
4. Kkma

Mecab is faster than Kkma about 30~40 times
Mecab is faster than Okt about 10 times
Mecab is faster than Komoran about 5 times
"""

# Early versions use okt and macab.
mecab = Mecab()
okt = Okt()

""" 
ERROR 3. ParserError: Error tokenizing data. C error 
- Solution: add code in read_csv = , sep='\t'
"""
test = pd.read_csv('KKH_20200129~20210707.txt', sep='\t')

# 1. Data extraction
x_list = []
y_list = []
data_extraction(x_list, y_list)

# 2. Space cailbrating
calibrate_spcaing(y_list)
calibrate_spcaing(x_list)

# 3. Modeling
model = Sequential()